<a href="https://www.nvidia.com/dli"> <img src="images/DLI Header.png" alt="Header" style="width: 400px;"/> </a>

***

# 2.0 ROS Integration of Image Recognition

In this section, you will take a step back from the ROS environment and build a deep learning image recognition model with NVIDIA DIGITS, an interactive _training_ system designed for common DNN (Deep Neural Network) training tasks. Models can be built in a variety of ways and deployed to an NVIDIA Jetson to run _inference_ on the robot with the onboard GPU. After expanding your understanding of models and how to buid them, you will integrate one of the pre-trained deployed models into this robot's ROS network. The `imagenet` ROS node from the `ros_deep_learning package` is used to wrap the inference data. You'll test your code in simulation and then deploy it to the physical robot.

[2.1 What is Image Recognition?](#image_recognition)<br>
[2.2 Image Recognition Models](#models)<br>
[2.3 Inference on the Robot's Jetson](0230_InferenceOnJetson.ipynb#inference)<br>
[2.4 Integrating Inference into ROS](0230_InferenceOnJetson.ipynb#integration)<br>
[2.5 Your ROS Image Recognition node: bot-nav classifier](0230_InferenceOnJetson.ipynb#simulation)<br>
[2.6 Assessment](0230_InferenceOnJetson.ipynb#assessment)<br>
[2.7 Deployment](0230_InferenceOnJetson.ipynb#deployment)<br>
[2.8 Recap](0230_InferenceOnJetson.ipynb#recap)<br>
[Appendix](9900_DL4R_Appendix.ipynb#apx-2)

<a id='image_recognition'></a>
# 2.1 What is Image Recognition with Deep Learning?

### Deep Learning - Amazing Achievements in AI

One of the most remarkable disruptions happening in today’s world is the emergence of modern AI by way of Deep Learning. Deep Learning is a neural-network-based approach to machine learning. What distinguishes it from previous approaches is that it enables you to train with very large datasets. This results in much more accurate performance. Early work in this area started with things like image recognition (also called image classification) and natural language processing, but has now expanded to far more advanced applications.

<img src="images/deeplearning.png" alt="achievements-in-AI" width=800/>

There are three advancements that have made all of these applications possible. First is the existence of large datasets for training (<B>big data</B>). With the proliferation of cameras and big data solutions, there is simply more data for training neural networks. The second is new <B>algorithms</B> that train the networks far more efficiently. Finally, the third is the availability of <B>GPUs</B>. The new algorithms are parallel in nature, which is perfectly suited to the highly parallelized GPU architectures.

### Training and Inference
In the past, a computer vision Ph.D. might spend months hand-tuning algorithms to detect an object (such as a cat) in an image. With a Deep Learning model, we simply train the neural network with many examples of the data that we want it to learn recognize. In _supervised learning_, we tell the network what is in the training data, using _labels_. Show it lots of cat pictures labeled "cat".  Show it lots of dog pictures labeled "dog". Do this until the network can reliably tell the difference between a picture of a cat and picture of a dog without the labels.  Through this <B>training</B> process, the network learns to _classify_ the images as one label or the other ("cat" or "dog").  This is the central feature of **Image Recognition** (also called **Image Classification**).

The result of training is a _trained model_ that can be deployed for the second part of Deep Learning, which is called <B>inference</B>. In inference, we show the network new data that it has not seen before, and the neural network makes inferences about that data. Show it a new cat, it identifies the image as a cat. Show it a new dog and it identifies the image as a dog, even if it's not the same dog it has seen before.

<img src="images/training-inference.png" alt="training-vs-inference" width=800/>

Today, most inferencing happens in a data center. Web apps and digital assistants use inferencing tor speech recognition and image recognition in the cloud.  <BR>
However, there are some use cases, like cars, robots, and UAVs, where inferencing must happen onboard, or **"at the edge"**.

Why is AI at the edge so important?<br>
It's simple; there are many use cases (such as robotics) where you simply cannot rely on a round trip through the cloud to solve the problem. NVIDIA built Jetson for solving problems that require AI at the edge. 

###  Deep Learning Workflow: Train and Deploy

Training a neural network model is very compute-resource intensive.  The most practical approach is to train neural network models on a workstation or server with GPUs. Once the model is trained, the trained model is exported to the Jetson so that inference can be run in the field, or "at the edge".

Here is the typical workflow; Train on a big fast computer and deploy the trained network to a Jetson.

<img src="images/workflow-train-deploy.png" alt="workflow-train-deploy" width=800/>


<a id='models'></a>
# 2.2 Image Recognition Models

We're going to train our own image classification model using DIGITS.

The goal in this example is to identify/read hand-written digits using the famous [MNIST](http://yann.lecun.com/exdb/mnist/) dataset. This data comes from the dataset which was created by Yann LeCun. It's largely considered the "Hello World," or introduction, to deep learning and image recognition.

You will use NVIDIA DIGITS to train an image recognition model on a host machine. For this lab, there is a DIGITS instance running with a GPU in the cloud, and you can access it through a browser window.

### <a href="/digits/" target="_blank">OPEN DIGITS</a>
First, click the link above to launch DIGITS.<br>
Keep both browser windows open (this notebook and DIGITS) so that you can refer to these instructions as you use the DIGITS tool. 

This exercise is based on <a href="https://github.com/NVIDIA/DIGITS/blob/master/docs/GettingStarted.md" target="_blank">this DIGITS tutorial ("Getting Started").</a><br>
It includes:
- Dataset creation phase, and
- Model creation and training phase

### Dataset
To begin, you need to provide a "dataset" on DIGITS, for training a neural network model.  You can manually collect many images for your task, or you can use a public dataset already prepared for research purposes.  For hand-written digits images, there is the famous MNIST dataset. 

For this lab, the MNIST data has already been downloaded and prepared under the following directory on the DIGITS host.

`/dli/data/train_invert`

Let's examine its structure.

In [ ]:
!ls -l /dli/data/train_invert

Note that there are directories/folders for each digit.  This are the "labels" the network will use to associate the images with a particular classification. In the case of numerical digits, there are 10 classifications, one per digit.

Let's also check what's inside each directory.

In [ ]:
!ls /dli/data/train_invert/0

This is how images are organized for an image classification network on DIGITS.

### Constructing a Dataset on DIGITS

Now on the other browser window that shows the DIGITS web UI, perform the following operations to construct the dataset. 

Begin by selecting the **Datasets** tab on the left.

<img src="images/digits_home_datasets.png" alt="DIGITShome" width=800/>

Since we want our network to tell us which "class" each image belongs to, we ask DIGITS to prepare a "classification" image dataset by selecting "Classification" from the "Images" menu on the right.

At this point, you may need to enter a username.  If requested, just enter any name in lower-case.

### Loading and organizing our data

You'll see that you've got a lot of options around *how* to load a dataset. For this first run-through, we're going to simplify and only fill out two fields. 

1. Copy and paste the following filepath into the field "Training Images":  <code>/dli/data/train_invert</code>
2. Change the image size from the default 256X256 Color to 28X28 Grayscale
3. Name the dataset so that you can find it. You can name **"mnist"**

Don't see "Training Images?" Click "DIGITS" on the top left and select "Datasets" before selecting "Images" and "Classification."

<img src="images/digits_new_dataset_dli_data.png" alt="datasettings" width=800/>

Then press `Create`.

DIGITS is now creating your dataset from the folder. Inside the folder, <code>train_invert</code>, there are 10 sub-folders, one for each class (0, 1, 2, 3, ..., 9). All of the handwritten training images of '0's are in the '0' folder, '1's are in the '1' folder, etc.  

Explore what our data looks like by selecting "Explore the DB".

### Your data

While there is an endless amount of analysis that we could do on the data, make sure you at least note the following:

1. This data is *labeled.* Each image in the dataset is paired with a **label** that informs the network what number the image represents, 0-9. We're basically providing a question with its answer, or, as our network will see it, the desired output with each input. These are the "examples" that our network will learn from.
2. Each image is simply a digit on a plain background. Image classification is the task of identifying the *predominant* object in an image. For a first attempt, we're using images that only contain *one* object. 

### Build a Model 

On the DIGITS web UI, click "DIGITS" on the top left of the screen to go home.

Creating a new model in DIGITS is a lot like creating a new dataset. From the home screen, the "Models" tab will be pre-selected. Click "Images" under "New Model" and select "Classification", as we're creating an image classification model to match our image classification dataset and image classification task.

<img src="images/digits_home_models.png" alt="newmodel" width=800/>

Again, for this first round of training let's keep it simple. The following are the fewest settings you could possibly set to successfully train a network.

1. We need to choose the dataset we just created. Select our **"mnist"** dataset.
2. We need to tell the network how long we want it to train. An **epoch** is one trip through the entire training dataset. Set the number of **Training Epochs** to 5 to give our network enough time to learn something, but not take all day. In general, this is a great setting to experiment with. 
3. We need to define which network will *learn* from our data. Select the network **LeNet** because it was designed to classify digits.
4. We need to name the model. You can name it **"LeNet_mnist"**.

<img src="images/build_a_model_mnist.png" alt="jetsonmodel" width=800/>

When you have set all of these options, press the Create button.  

You are now training your model! For this configuration, the model training should complete in less than 1 minute. When done, the Job Status on the right will say "Done." Next, let's go back to the deployment part of this lab by closing this tab.

### Training

While training you will see a screen like this.<br>
You should find the expected completion time on the right side.

<img src="images/lenet_model_training.png" alt="classifyimage" width=800/>

Finally, test the trained model against an image.  Scroll down to the bottom of the page and select an image from the `/dli/data/test_invert/5/` path (any will do!).  Check the "Show visualizations and statistics" checkbox and click the "Classify One" button.

<img src="images/classify-one-image.png" alt="classifyimage" width=800/>

Congratulations!  You've successfully trained a classification deep learning model for image recognition and tested the model on an unknown image!<br>
When your model classifies an unknown image, it is performing the **inference** process.  So far, this inference is occurring only on the DIGITS machine that is in the cloud.

Next, we will see how the trained model can be deployed so that the inference process happens on our robot.

<a id='inference'></a>
## [--> Next: 2.3 Inference on the Robot's Jetson](0230_InferenceOnJetson.ipynb)

<div style="border:2px solid black; background-color:#f2f2f2; padding:10px; width:400px; margin:auto;">
<b><a href="DLI Welcome Notebook.ipynb">Overview</a></b><br>
<b><a href="0100_DLforRoboticsControl.ipynb">1.0 Introduction to ROS Control</a></b><br>
<b><a href="0200_DLforRoboticsImageRecognition.ipynb">2.0 ROS Integration of Image Recognition</a></b><br>
<b><a href="0300_DLforRoboticsObjectDetection.ipynb">3.0 ROS Integration of Object Detection</a></b><br>
<b><a href="9900_DL4R_Appendix.ipynb">Appendix</a></b><br>
</div>

<a href="https://www.nvidia.com/dli"> <img src="images/DLI Header.png" alt="Header" style="width: 400px;"/> </a>